In [183]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [73]:
final_test = pd.read_csv("../data/processed/final_test.csv")
final_train = pd.read_csv("../data/processed/final_train.csv")

In [74]:
final_train.head()

,Survived,Age,Fare,TravelAlone,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Sex_male,IsMinor
0,0,22.0,7.2500,0,0,0,1,0,0,1,1,0
1,1,38.0,71.2833,0,1,0,0,1,0,0,0,0
2,1,26.0,7.9250,1,0,0,1,0,0,1,0,0
3,1,35.0,53.1000,0,1,0,0,0,0,1,0,0
4,0,35.0,8.0500,1,0,0,1,0,0,1,1,0


In [75]:
final_test.head()

,PassengerId,Age,Fare,TravelAlone,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Sex_male,IsMinor
0,892,34.5,7.8292,1,0,0,1,0,1,0,1,0
1,893,47.0,7.0000,0,0,0,1,0,0,1,0,0
2,894,62.0,9.6875,1,0,1,0,0,1,0,1,0
3,895,27.0,8.6625,1,0,0,1,0,0,1,1,0
4,896,22.0,12.2875,0,0,0,1,0,0,1,0,0


In [9]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [80]:
feature_columns_to_use = ['Age','TravelAlone','Pclass_1','Pclass_2', 'Fare','Sex_male', 'IsMinor', 'Embarked_C', 'Embarked_S']

In [81]:
final_train.head()

,Survived,Age,Fare,TravelAlone,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S,Sex_male,IsMinor
0,0,22.0,7.2500,0,0,0,1,0,0,1,1,0
1,1,38.0,71.2833,0,1,0,0,1,0,0,0,0
2,1,26.0,7.9250,1,0,0,1,0,0,1,0,0
3,1,35.0,53.1000,0,1,0,0,0,0,1,0,0
4,0,35.0,8.0500,1,0,0,1,0,0,1,1,0


In [82]:
# Prepare the inputs for the model
y = final_train['Survived']
X = final_train.loc[:, feature_columns_to_use]

train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=104, 
                                                           test_size=0.25, 
                                                           shuffle=True)


In [99]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

In [40]:
from sklearn import metrics

# Model grid search optimiser

In [137]:
# define model
model = xgb.XGBClassifier()
# define search space
space = dict(max_depth=[10, 11, 12], n_estimators=[220,230,240], learning_rate=[0.025, 0.05, 0.075])

In [138]:
# define search
search = GridSearchCV(model, space)

In [139]:
# execute search
result = search.fit(X, y)

# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: 0.8260686711443098
Best Hyperparameters: {'learning_rate': 0.025, 'max_depth': 12, 'n_estimators': 230}


# Apply results to model

In [175]:
# You can experiment with many other options here, using the same .fit() and .predict()
# methods; see http://scikit-learn.org
# This example uses the current build of XGBoost, from https://github.com/dmlc/xgboost
gbm = xgb.XGBClassifier(max_depth=150, n_estimators=250, learning_rate=0.000125).fit(train_X, train_y)
predictions = gbm.predict(test_X)

In [176]:
results = metrics.confusion_matrix(test_y, predictions)
print(results)

[[119  26]
 [ 23  55]]


In [177]:
#print accuracy of model
print(metrics.accuracy_score(y_actual, y_predicted))

#print precision value of model
print(metrics.precision_score(y_actual, y_predicted))

#print recall value of model
print(metrics.recall_score(y_actual, y_predicted))

0.757847533632287
0.6428571428571429
0.6923076923076923


In [178]:
print(metrics.f1_score(y_actual, y_predicted))

0.6666666666666666


In [179]:
y_actual = pd.Series(test_y, name='Actual')
y_predicted = pd.Series(predictions, name='Predicted')

#create confusion matrix
print(pd.crosstab(y_actual, y_predicted))

Predicted   0   1
Actual           
0          28  10
1          14   9


# Make predictions to submit

In [180]:
final_X = final_test.loc[:, feature_columns_to_use]

In [181]:
predictions = gbm.predict(final_X)

In [182]:
# Kaggle needs the submission to have a certain format;
# see https://www.kaggle.com/c/titanic-gettingStarted/download/gendermodel.csv
# for an example of what it's supposed to look like.
submission = pd.DataFrame({ 'PassengerId': final_test['PassengerId'],
                            'Survived': predictions })
submission.to_csv("../data/processed/submission.csv", index=False)